In [1]:
import pandas as pd
import numpy as np
import random
import os
import torch
import sys
sys.path.append('../')
from modules import utils, constants
import warnings
warnings.filterwarnings('ignore')
from stable_baselines import DQN
from stable_baselines import bench, logger
# from stable_baselines.common.vec_env import DummyVecEnv, VecNormalize
import tensorflow
from stable_baselines.common.callbacks import CheckpointCallback
# from modules.env import LupusEnv

Invalid MIT-MAGIC-COOKIE-1 key

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tensorflow.set_random_seed(constants.SEED)

#### The data

In [3]:
# train_df = pd.read_csv('../data/25_jan/train_set_basic.csv')
train_df = pd.read_csv('../data/missingness/0/training_set.csv')
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


In [4]:
train_df.cutaneous_lupus.value_counts()

0    44200
3     3120
1     2246
2      834
Name: cutaneous_lupus, dtype: int64

In [5]:
train_df[(train_df.ana==0) & (train_df.label==1)]

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label


In [6]:
train_df.isna().sum()

ana                             0
fever                           0
leukopenia                      0
thrombocytopenia                0
auto_immune_hemolysis           0
delirium                        0
psychosis                       0
seizure                         0
non_scarring_alopecia           0
oral_ulcers                     0
cutaneous_lupus                 0
pleural_effusion                0
pericardial_effusion            0
acute_pericarditis              0
joint_involvement               0
proteinuria                     0
anti_cardioliphin_antibodies    0
anti_β2gp1_antibodies           0
lupus_anti_coagulant            0
low_c3                          0
low_c4                          0
anti_dsdna_antibody             0
anti_smith_antibody             0
label                           0
dtype: int64

In [7]:
len(train_df)

50400

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 23), (50400,))

In [9]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No lupus',
 'Lupus',
 'Inconclusive diagnosis',
 'ana',
 'fever',
 'leukopenia',
 'thrombocytopenia',
 'auto_immune_hemolysis',
 'delirium',
 'psychosis',
 'seizure',
 'non_scarring_alopecia',
 'oral_ulcers',
 'cutaneous_lupus',
 'pleural_effusion',
 'pericardial_effusion',
 'acute_pericarditis',
 'joint_involvement',
 'proteinuria',
 'anti_cardioliphin_antibodies',
 'anti_β2gp1_antibodies',
 'lupus_anti_coagulant',
 'low_c3',
 'low_c4',
 'anti_dsdna_antibody',
 'anti_smith_antibody']

#### The Model

In [ ]:
training_env = utils.create_env(X_train, y_train)
training_env = bench.Monitor(training_env, logger.get_dir())

model = DQN('MlpPolicy', training_env, verbose=1, seed=constants.SEED, learning_rate=0.0001, buffer_size=1000000, 
            learning_starts=50000, train_freq=4, target_network_update_freq=10000, exploration_final_eps=0.05, 
            n_cpu_tf_sess=1, policy_kwargs=dict(dueling=False), double_q=False)

    
checkpoint_callback = CheckpointCallback(save_freq=100000, save_path='../models/logs/sb/vanilla_dqn', 
                                         name_prefix='ddqn_basic')

model.learn(total_timesteps=200000000, log_interval=100000, callback=checkpoint_callback)

# Save the trained DQN agent
model.save('../models/22_mar/vanilla_dqn_lupus_diagnosis')
# training_env.close()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.
--------------------------------------
| % time spent exploring  | 97       |
| episodes                | 100000   |
| mean 100 episode reward | -0.8     |
| steps                   | 466816   |
| success rate            | 0.16     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 200000   |
| mean 100 episode reward | -0.7     |
| steps                   | 939089   |
| success rate            | 0.2      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 300000   |
| mean 100 episode reward | -0.7     |
| steps                   | 1419596  |
| success rate            | 0.19     |
--------------------------------------


--------------------------------------
| % time spent exploring  | 7        |
| episodes                | 2800000  |
| mean 100 episode reward | -0.8     |
| steps                   | 19572832 |
| success rate            | 0.33     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2900000  |
| mean 100 episode reward | -0.7     |
| steps                   | 20972742 |
| success rate            | 0.33     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3000000  |
| mean 100 episode reward | -0.6     |
| steps                   | 22210456 |
| success rate            | 0.41     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3100000  |
| mean 100 episode reward | -0.5     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 5900000  |
| mean 100 episode reward | -0.5     |
| steps                   | 61708036 |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6000000  |
| mean 100 episode reward | -0.6     |
| steps                   | 63114366 |
| success rate            | 0.42     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6100000  |
| mean 100 episode reward | -0.7     |
| steps                   | 64500837 |
| success rate            | 0.35     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6200000  |
| mean 100 episode reward | -0.5     |
| steps                  

---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 8900000   |
| mean 100 episode reward | -0.7      |
| steps                   | 105713710 |
| success rate            | 0.37      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 9000000   |
| mean 100 episode reward | -0.4      |
| steps                   | 107221695 |
| success rate            | 0.5       |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 9100000   |
| mean 100 episode reward | -0.7      |
| steps                   | 108753148 |
| success rate            | 0.41      |
---------------------------------------
---------------------------------------
| % time spent exploring  | 5         |
| episodes                | 9200000   |
| mean 100 episode reward | -0.7      |
